In [ ]:
!pip install transformers
!pip install sentence-transformers
!pip install langchain

In [3]:
from langchain import HuggingFaceHub
import os
from dotenv import load_dotenv
import pdfplumber

In [11]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

def read_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text() + '\n'
    return text

file_path = 'ArtificialIntelligenceAct-1-50.pdf'  
document_text = read_pdf(file_path)

paragraphs = document_text.split('\n') 
print(len(paragraphs))
with open('paragraphs.txt', 'w', encoding='utf-8') as f:
    for par in paragraphs:
        f.write(par + '\n\n\n')

embeddings = model.encode(paragraphs)

with open('embeddings.txt', 'w') as f:
    for i, embedding in enumerate(embeddings):
        f.write(f"Paragraph {i+1} embeddings: {embedding.tolist()}\n")

1009
